# 3D Surface detection

An custom implementation of Baehnisch et. al (2009): "Fast and Accurate 3D Edge Detection for Surface Reconstruction".

---

## 0. Environmental setup

In [67]:
import numpy as np
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
import tifffile
import sys

sys.path.append('..')

from src.utils.io import get_file_list

In [68]:
def show_plane(ax, plane, cmap="gray", title=None):
    ax.imshow(plane, cmap=cmap)
    ax.axis("off")

    if title:
        ax.set_title(title)

        
def explore_slices(data, cmap="gray"):
    from ipywidgets import interact
    N = len(data)

    @interact(plane=(0, N - 1))
    def display_slice(plane=34):
        fig, ax = plt.subplots(figsize=(20, 5))

        show_plane(ax, data[plane], title="Plane {}".format(plane), cmap=cmap)

        plt.show()

    return display_slice

def explore_slices_2_samples(data, cmap="gray"):
    from ipywidgets import interact
    N = len(data[0])

    @interact(plane=(0, N - 1))
    def display_slice(plane=34):
        fig, ax = plt.subplots(figsize=(20, 5), nrows=1, ncols=2)

        show_plane(ax[0], data[0][plane], title="Plane {}".format(plane), cmap=cmap)
        show_plane(ax[1], data[1][plane], title='Plane {}'.format(plane), cmap=cmap)

        plt.show()

    return display_slice


---

## 1. Read in data

We will load a number of 3D images in order to test our following implementation of 3D surface detection. To this end, we will again use some 3D images of T-cell nuclei.

In [69]:
root_dir = '../data/tcell_project/filtered/selected_slices_ar_08_area25_50_16bit'
file_list = get_file_list(root_dir)

cell_ids = list(range(20))
cells = []

for i in range(len(cell_ids)):
    cells.append(np.squeeze(tifffile.imread(file_list[cell_ids[i]])))

Let us have a look at those samples. We see that the data is quite noisy and the contrasts are not adjusted neither. However, for the purpose of testing our implementation of a 3D surface detection algorithm as described by Baehnisch et.al (2009) those images should suffice.

In [70]:
explore_slices_2_samples(cells)

interactive(children=(IntSlider(value=20, description='plane', max=20), Output()), _dom_classes=('widget-inter…

<function __main__.explore_slices_2_samples.<locals>.display_slice(plane=34)>

---
## 2. 3D Surface detection

### 2a. Smoothing

The first step is an initial smoothing of the image in order to get rid of the Noise in the image. To this end, we will apply 3D Gaussian smoothing.

Let us for now look at one cell and remove the first three and last three layers that seem to have a very low SNR.

In [71]:
cell = cells[0][2:-3]
cell = cell/cell.max() * 255
sigma = 1

smoothed_cell = ndi.gaussian_filter(cell, sigma)

Let us quickly inspect the result of these transformation to validate them.

In [72]:
explore_slices_2_samples([cell, smoothed_cell])

interactive(children=(IntSlider(value=15, description='plane', max=15), Output()), _dom_classes=('widget-inter…

<function __main__.explore_slices_2_samples.<locals>.display_slice(plane=34)>

### 2b. Gradient calculation

We will approximate the gradients using the Sobel operator.

In [73]:
dx = ndi.sobel(smoothed_cell, 2) 
dy = ndi.sobel(smoothed_cell, 1)
dz = ndi.sobel(smoothed_cell, 0)

Let us now calculate the magnitude of the gradients.

In [74]:
gradient = np.sqrt(dx**2 + dy**2 + dz**2)

Let us have a look at the gradients magnitude. We can already see different edges that are detected and that in 3D form consecutive surfaces.

In [75]:
explore_slices(gradient)

interactive(children=(IntSlider(value=15, description='plane', max=15), Output()), _dom_classes=('widget-inter…

<function __main__.explore_slices.<locals>.display_slice(plane=34)>

### 2c. Angle calculation

However, we see that the edges are still not well separated and are rather thick. For that purpose we will apply non-maximum suppression as in the basic Canny edge detection algorithm. The first step thereby is calculating the angles for the gradients.

In [76]:
theta = np.arctan2(dy, dx)
psi = np.arctan2(dz, dx)

# Convert to degree
theta = 180 + (180/np.pi)*theta
psi = 180 + (180/np.pi)*psi

In [77]:
xy_e = (theta < 22.5) + (theta >= 337.5) + (theta >= 112.5) * (theta < 202.5)
xy_ne = (theta >= 22.5) * (theta < 67.5) + (theta >= 202.5) * (theta < 247.5)
xy_n = (theta >= 67.5) * (theta < 112.5) + (theta >= 247.5) * (theta < 292.5)
xy_se = ~(xy_e + xy_ne + xy_n)

In [78]:
xz_e = (psi < 22.5) + (psi >= 337.5) + (psi >= 112.5) * (psi < 202.5)
xz_ne = (psi >= 22.5) * (psi < 67.5) + (psi >= 202.5) * (psi < 247.5)
xz_n = (psi >= 67.5) * (psi < 112.5) + (psi >= 247.5) * (psi < 292.5)
xz_se = ~(xz_e + xz_ne + xz_n)

In [79]:
# Encode the angles: 
# z,y,x


angles = np.zeros_like(gradient)
angles[xy_e * xz_e] = 0
angles[xy_e * xz_ne] = 1
angles[xy_e * xz_n] = 2
angles[xy_e * xz_se] = 3
angles[xy_ne * xz_e] = 4
angles[xy_ne * xz_ne] = 5
angles[xy_ne * xz_n] = 6
angles[xy_ne * xz_se] = 7
angles[xy_n * xz_e] = 8
angles[xy_n * xz_ne] = 9
angles[xy_n * xz_n] = 10
angles[xy_n * xz_se] = 11
angles[xy_se * xz_e] = 12
angles[xy_se * xz_ne] = 13
angles[xy_se * xz_n] = 14
angles[xy_se * xz_se] = 15

In [80]:
explore_slices(angles)

interactive(children=(IntSlider(value=15, description='plane', max=15), Output()), _dom_classes=('widget-inter…

<function __main__.explore_slices.<locals>.display_slice(plane=34)>

### 2d. Non-maximum suppression

In [81]:
def non_maximum_suppression_3d(gradient, angles, thresh = 300):
# E-E: 0 --> (0,0,1) and (0,0,-1)
# E-NE: 1 --> (-1,0,-1) and (1,0,1)
# E-N: 2 --> (-1,0,0) and (1,0,0)
# E-SE: 3 --> (-1,0,1) and (1,0,-1)
# NE-E: 4 --> (0,-1,1) and (0,1,-1)
# NE-NE: 5 --> (1,-1,1) and (-1,1,-1)
# NE-N: 6 --> (-1,0,0) and (1,0,0)
# NE-SE: 7 --> (-1,-1,1) and (1,1,-1)
# N-E: 8 --> (0,-1,0) and (0,1,0)
# N-NE: 9 --> (1,-1,0) and (-1,1,0)
# N-N: 10 --> (-1,0,0) and (1,0,0)
# N-SE: 11 --> (-1,-1,0) and (1,1,0)
# SE-E: 12 --> (0,1,1) and (0,-1,-1)
# SE-NE: 13 --> (1,1,1) and (-1,-1,-1)
# SE-N: 14 --> (-1,0,0) and (1,0,0)
# SE-SE: 15 --> (-1,1,1) and (1,-1,-1)
    result = np.zeros_like(gradient).astype(bool)
    depth, height, width = gradient.shape
    # i= z, j=y, k=x
    for z in range(depth):
        for y in range(height):
            for x in range(width):
                if gradient[z,y,x] < thresh:
                    continue
                elif z in [0,depth-1] or y in [0,height-1] or x in [0,width-1]:
                    continue
                elif angles[z,y,x] == 0:
                    ## E-E direction
                    n1 = z, y, x-1
                    n2 = z, y, x+1
                    result[z,y,x] = gradient[z,y,x] >= max(gradient[n1], gradient[n2])
                elif angles[z,y, x] == 1:
                    ## E-NE direction
                    n1 = z+1, y, x+1
                    n2 = z-1, y, x-1
                    result[z,y,x] = gradient[z,y,x] >= max(gradient[n1], gradient[n2])
                elif angles[z,y,x] == 2:
                    ## E-N direction
                    n1 = z-1, y, x
                    n2 = z+1, y, x
                    result[z,y,x] = gradient[z,y,x] >= max(gradient[n1], gradient[n2])
                elif angles[z,y,x] == 3:
                    ## E-SE direction
                    n1 = z-1, y, x+1
                    n2 = z+1, y, x-1
                    result[z,y,x] = gradient[z,y,x] >= max(gradient[n1], gradient[n2])
                elif angles[z,y,x] == 4:
                    ## NE-E direction
                    n1 = z, y-1, x+1
                    n2 = z, y+1, x-1
                    result[z,y,x] = gradient[z,y,x] >= max(gradient[n1], gradient[n2])
                elif angles[z,y,x] == 5:
                    ## NE-NE direction
                    n1 = z+1, y-1, x+1
                    n2 = z-1, y+1, x-1
                    result[z,y,x] = gradient[z,y,x] >= max(gradient[n1], gradient[n2])
                elif angles[z,y,x] == 6:
                    ## NE-N direction
                    n1 = z-1, y, x
                    n2 = z+1, y, x
                    result[z,y,x] = gradient[z,y,x] >= max(gradient[n1], gradient[n2])
                elif angles[z,y,x] == 7:
                    ## NE-SE direction
                    n1 = z-1, y-1, x+1
                    n2 = z+1, y+1, x-1
                    result[z,y,x] = gradient[z,y,x] >= max(gradient[n1], gradient[n2])
                elif angles[z,y,x] == 8:
                    ## N-E direction
                    n1 = z, y-1, x
                    n2 = z, y+1, x
                    result[z,y,x] = gradient[z,y,x] >= max(gradient[n1], gradient[n2])
                elif angles[z,y,x] == 9:
                    ## N-NE direction
                    n1 = z+1, y-1, x
                    n2 = z-1, y+1, x
                    result[z,y,x] = gradient[z,y,x] >= max(gradient[n1], gradient[n2])
                elif angles[z,y,x] == 10:
                    ## N-N direction
                    n1 = z-1, y, x
                    n2 = z+1, y, x
                    result[z,y,x] = gradient[z,y,x] >= max(gradient[n1], gradient[n2])
                elif angles[z,y,x] == 11:
                    ## N-SE direction
                    n1 = z-1, y-1, x
                    n2 = z+1, y+1, x
                    result[z,y,x] = gradient[z,y,x] >= max(gradient[n1], gradient[n2])
                elif angles[z,y,x] == 12:
                    ## SE-E direction
                    n1 = z, y-1, x-1
                    n2 = z, y+1, x+1
                    result[z,y,x] = gradient[z,y,x] >= max(gradient[n1], gradient[n2])
                elif angles[z,y,x] == 13:
                    ## SE-NE direction
                    n1 = z-1, y-1, x-1
                    n2 = z+1, y+1, x+1
                    result[z,y,x] = gradient[z,y,x] >= max(gradient[n1], gradient[n2])
                elif angles[z,y,x] == 14:
                    ## SE-N direction
                    n1 = z-1, y, x
                    n2 = z+1, y, x
                    result[z,y,x] = gradient[z,y,x] >= max(gradient[n1], gradient[n2])
                else:
                    ## SE-SE direction
                    n1 = z-1, y+1, x+1
                    n2 = z+1, y-1, x-1
                    result[z,y,x] = gradient[z,y,x] >= max(gradient[n1], gradient[n2])
    return result

In [82]:
###### Non-maximum suppression ########
def non_maximum_suppression_2d(gradient, angles, thresh):
    w, h = gradient.shape
    mask = np.zeros_like(gradient).astype(bool)
    for i in range(w):
        for j in range(h):
            if gradient[i,j] <= thresh:
                mask[i,j] = False
            elif i in [0, w-1] or j in [0, h-1]:
                mask[i,j] = False
            elif angles[i, j] == 0:
                #east direction
                n1 = i, j-1
                n2 = i, j+1
                mask[i,j] = gradient[i,j] >= max(gradient[n1], gradient[n2])
            elif angles[i, j] == 1:
                #north east direction
                n1 = i-1, j+1
                n2 = i+1, j-1
                mask[i,j] = gradient[i,j] >= max(gradient[n1], gradient[n2])
            elif angles[i, j] == 2:
                # north direction
                n1 = i-1, j
                n2 = i+1, j
                mask[i,j] = gradient[i,j] >= max(gradient[n1], gradient[n2])
            else:
                # south east direction
                n1 = i-1, j-1
                n2 = i+1, j+1
                mask[i,j] = gradient[i,j] >= max(gradient[n1], gradient[n2])
    return mask

In [83]:
nms = non_maximum_suppression_3d(gradient, angles, 330)

In [84]:
angles_2d = np.zeros_like(gradient)
angles_2d[xy_e] = 0
angles_2d[xy_ne] = 1
angles_2d[xy_n] = 2
angles_2d[xy_se] = 3

In [87]:
explore_slices_2_samples([nms, gradient])

interactive(children=(IntSlider(value=15, description='plane', max=15), Output()), _dom_classes=('widget-inter…

<function __main__.explore_slices_2_samples.<locals>.display_slice(plane=34)>

In [86]:
from skimage.filters import threshold_otsu
explore_slices(gradient >threshold_otsu(gradient))

interactive(children=(IntSlider(value=15, description='plane', max=15), Output()), _dom_classes=('widget-inter…

<function __main__.explore_slices.<locals>.display_slice(plane=34)>